In [5]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

In [6]:
import xgboost as xgb
from xgboost import XGBClassifier

In [2]:
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN

In [7]:
train_data = pd.read_csv('HW4 - training data.csv', sep = ';')
test_data = pd.read_csv('HW4 - test data.csv',sep = ';')

In [5]:
X_train_sample = train_data.iloc[:,[0,1,2,6,7,8,9,10,11,12,13,24]]
Y_train_sample = train_data['adopter']
X_test_sample  = test_data.iloc[:,[0,1,2,6,7,8,9,10,11,12,13,24]]
X_train_sample, X_validation_sample, Y_train_sample, Y_validation_sample = train_test_split(X_train_sample, Y_train_sample, test_size = 0.3)

In [8]:
X_train = train_data.iloc[:,0:25]
Y_train = train_data['adopter']
X_test  = test_data.iloc[:,0:25]
X_train, X_validation, Y_train, Y_validation = train_test_split(X_train, Y_train, test_size = 0.3)

In [7]:
train_data.shape[0]/train_data[train_data['adopter']  == 1].shape[0]

56.287012987012986

In [10]:
ros = RandomOverSampler(random_state = 0)
X_train_resampled, Y_train_resampled = ros.fit_sample(X_train, Y_train)

In [32]:
X_train_resampled, Y_train_resampled = SMOTE().fit_sample(X_train, Y_train)

In [25]:
X_train_resampled, Y_train_resampled = ADASYN().fit_sample(X_train, Y_train)

In [10]:
from imblearn.ensemble import EasyEnsemble

In [12]:
ee = EasyEnsemble(random_state = 0, n_subsets = 10)

In [13]:
X_train_ensemble, Y_train_ensemble = ee.fit_sample(X_train, Y_train)

In [22]:
X_train_ensemble.shape

(10, 2148, 25)

In [25]:
Y_train_ensemble[0].shape

(2148,)

In [21]:
from sklearn.svm import LinearSVC


In [26]:
svc = LinearSVC()
pred_ee = pd.DataFrame()
for i in range(10):
    svc.fit(X_train_ensemble[i],Y_train_ensemble[i])
    pred_ee[i] = svc.predict(X_validation)

In [32]:
np.sum(Y_validation)

466

In [17]:
svc_original = LinearSVC()
svc_original.fit(X_train, Y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [43]:
f1_score(y_pred=svc.predict(X_validation), y_true=Y_validation)

0.0692867540029112

In [49]:
X_validation.values

array([[21.,  0.,  2., ..., 36.,  1.,  0.],
       [29.,  0.,  1., ..., 60.,  1.,  0.],
       [17.,  0.,  1., ..., 29.,  0.,  0.],
       ...,
       [13.,  0.,  5., ..., 14.,  1.,  0.],
       [24.,  0., 18., ..., 54.,  0.,  0.],
       [21.,  1.,  1., ..., 35.,  0.,  0.]])

# XGBoost

In [3]:
from imblearn.ensemble import EasyEnsemble

In [55]:
ee = EasyEnsemble(n_subsets = 20)

In [56]:
x_resampled, y_resampled = ee.fit_sample(X_train, Y_train)

In [57]:
x_resampled.shape

(20, 2138, 25)

In [58]:
xgb_model =XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None, silent=1,
       subsample=0.8)

In [59]:
xgb_model.fit(x_resampled[4], y_resampled[4])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None, silent=1,
       subsample=0.8)

In [60]:
f1_score(y_pred=xgb_model.predict(np.array(X_validation)), y_true = Y_validation)


/home/chloe/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.08247422680412371

In [145]:
pred = pd.DataFrame()

In [146]:
for i in range(20):
    xgb_model.fit(x_resampled[i], y_resampled[i])
    pred[i] =  xgb_model.predict(np.array(X_test))
    print(i)

/home/chloe/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0


/home/chloe/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


1


/home/chloe/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


...

/home/chloe/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


17


/home/chloe/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


18
19


/home/chloe/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [147]:
ensemble = list()

In [148]:
for i in range(pred.shape[0]):
    if np.sum(pred.iloc[i,:])>19:
        ensemble.append(1)
    else:
        ensemble.append(0)

In [150]:
ensemble

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,


In [144]:
f1_score(y_pred=ensemble, y_true = Y_validation)


0.11370606275064873

In [152]:
submission = pd.DataFrame(test_data['user_id'])

submission['prediction(adopter)'] = ensemble

In [153]:
submission.to_csv('submission_ensemble.csv',index = False)